# Variable coefficient $p$-system with periodic BCs

In [ ]:
include("setup.jl")

## Compute the energy for one chosen setting

In [ ]:
# setup similar to https://github.com/clawpack/pyclaw/blob/master/examples/stegoton_1d/stegoton.py
# but with smooth ϱ, K
function ϱ_exp(x)
    2.5 - 1.5 * sinpi(2x)
end
K_exp(x) = ϱ_exp(x)
function σ_exp(ε::SymPy.Sym, x)
    exp(K_exp(x) * ε) - 1
end
function σ_exp(ε, x)
    expm1(K_exp(x) * ε)
end
function η_exp(ε, u, x)
    K = K_exp(x)
    exp(ε * K) / K - ε - inv(K) + ϱ_exp(x) * u^2 / 2
end

function doit(σ)
    ε, x, u = sp.symbols("varepsilon, x, u", real=true)
    
    antiderivative = sp.integrate(σ(ε, x), ε)
    antiderivative = antiderivative - antiderivative.subs(ε, 0)
    antiderivative + ϱ_exp(x) * u^2 / 2 - η_exp(ε, u, x) |> sp.simplify
end

doit(σ_exp)

## Discretization

In [ ]:
function relaxation_functional(γ, εu_new, εu_old, param)
    @unpack D, x, tmp1, η = param
    ε_new = εu_new.x[1]
    u_new = εu_new.x[2]
    ε_old = εu_old.x[1]
    u_old = εu_old.x[2]
    
    @. tmp1 = η((1-γ)*ε_old + γ*ε_new, (1-γ)*u_old + γ*u_new, x)
    energy = integrate(tmp1, D)
end
function relaxation_functional(εu, param)
    @unpack D, x, tmp1, η = param
    ε = εu.x[1]
    u = εu.x[2]
    
    @. tmp1 = η(ε, u, x)
    energy = integrate(tmp1, D)
end


function save_func_periodic(εu, t, integrator)
    @unpack D, x, tmp1, ϱ, η, σ, ε_sol, u_sol = integrator.p
    ε = εu.x[1]
    u = εu.x[2]
    print(".")
    
    mass_ε = integrate(ε, D)
    @. tmp1 = ϱ(x) * u
    mass_u = integrate(tmp1, D)
    energy = relaxation_functional(εu, integrator.p)
    
    @. tmp1 = (ε - ε_sol(t, x))^2
    error_ε = integrate(tmp1, D) |> sqrt
    
    @. tmp1 = (u - u_sol(t, x))^2
    error_u = integrate(tmp1, D) |> sqrt
    
    @. tmp1 = (σ(ε, x) - σ(ε_sol(t, x), x))^2
    error_σ = integrate(tmp1, D) |> sqrt
    
    @. tmp1 = (ϱ(x)*u - ϱ(x)*u_sol(t, x))^2
    error_ϱu = integrate(tmp1, D) |> sqrt
    
    SVector(mass_ε, mass_u, energy, error_ε, error_u, error_σ, error_ϱu)
end


function psystem_periodic!(dεu, εu, param, t)
    @unpack D, x, tmp1, σ, ϱ = param
    ε  = εu.x[1]
    u  = εu.x[2]
    dε = dεu.x[1]
    du = dεu.x[2]
    
    mul!(dε, D, u)
    @. tmp1 = σ(ε, x)
    mul!(du, D, tmp1)
    @. du = du / ϱ(x)
    
    nothing
end

function solve_ode_psystem_periodic(ε_sol, u_sol, D, σ, ϱ, η, tspan, alg, tol, dt, adaptive, num_periods)
    x = grid(D)
    ε0 = ε_sol.(tspan[1], x)
    u0 = u_sol.(tspan[1], x)
    εu0 = ArrayPartition(ε0, u0)
    tmp1 = similar(u0)
    param = (;D, x, tmp1, σ, ϱ, η, u_sol, ε_sol)

    ode = ODEProblem(psystem_periodic!, εu0, tspan, param)
    
    saveat = range(tspan..., length=num_periods+1)
    saved_values_baseline = SavedValues(eltype(D), SVector{7,eltype(D)})
    saving_baseline = SavingCallback(save_func_periodic, saved_values_baseline, saveat=saveat)
    saved_values_relaxation = SavedValues(eltype(D), SVector{7,eltype(D)})
    saving_relaxation = SavingCallback(save_func_periodic, saved_values_relaxation, saveat=saveat)
    relaxation = DiscreteCallback((u,t,integrator) -> true, relaxation!,
        save_positions=(false,false))
    cb_baseline = CallbackSet(saving_baseline)
    cb_relaxation = CallbackSet(relaxation, saving_relaxation)
    
    sol_baseline = solve(ode, alg, abstol=tol, reltol=tol, dt=dt, adaptive=adaptive, 
        callback=cb_baseline, dense=false,
        save_everystep=false, tstops=saveat, saveat=saveat)
    flush(stdout)
    sol_relaxation = solve(ode, alg, abstol=tol, reltol=tol, dt=dt, adaptive=adaptive,
        callback=cb_relaxation, dense=false,
        save_everystep=false, tstops=saveat, saveat=saveat)
    flush(stdout)

    ε_num_baseline   = sol_baseline[end].x[1]
    u_num_baseline   = sol_baseline[end].x[2]
    ε_num_relaxation = sol_relaxation[end].x[1]
    u_num_relaxation = sol_relaxation[end].x[2]
    ε_ana = ε_sol.(tspan[end], x)
    u_ana = u_sol.(tspan[end], x)
    @printf("Error in ε (baseline):   %.3e\n", integrate(u->u^2, ε_num_baseline   - ε_ana, D) |> sqrt)
    @printf("Error in ε (relaxation): %.3e\n", integrate(u->u^2, ε_num_relaxation - ε_ana, D) |> sqrt)
    @printf("Error in u (baseline):   %.3e\n", integrate(u->u^2, u_num_baseline   - u_ana, D) |> sqrt)
    @printf("Error in u (relaxation): %.3e\n", integrate(u->u^2, u_num_relaxation - u_ana, D) |> sqrt)
    @printf("Difference of baseline and relaxation in ε: %.3e\n", 
        integrate(u->u^2, ε_num_baseline - ε_num_relaxation, D) |> sqrt)
    @printf("Difference of baseline and relaxation in u: %.3e\n", 
        integrate(u->u^2, u_num_baseline - u_num_relaxation, D) |> sqrt)

    sleep(0.1)
    fig_ε, ax = plt.subplots(1, 1)
    plt.plot(x, ε0, label=L"\varepsilon^0")
    plt.plot(x, ε_num_baseline,   label=L"$\varepsilon^\mathrm{num}$ (non-conservative)")
    plt.plot(x, ε_num_relaxation, label=L"$\varepsilon^\mathrm{num}$ (conservative)")
    plt.xlabel(L"x"); plt.ylabel(L"\varepsilon")
    plt.legend(loc="center left", bbox_to_anchor=(1.0, 0.5));

    fig_σ, ax = plt.subplots(1, 1)
    plt.plot(x, σ.(ε0, x), label=L"\sigma^0")
    plt.plot(x, σ.(ε_num_baseline, x),   label=L"$\sigma^\mathrm{num}$ (non-conservative)")
    plt.plot(x, σ.(ε_num_relaxation, x), label=L"$\sigma^\mathrm{num}$ (conservative)")
    plt.xlabel(L"x"); plt.ylabel(L"\sigma")
    plt.legend(loc="center left", bbox_to_anchor=(1.0, 0.5));

    fig_u, ax = plt.subplots(1, 1)
    plt.plot(x, u0, label=L"u^0")
#     plt.plot(x, u_ana, label=L"$u^\mathrm{ana}$")
    plt.plot(x, u_num_baseline,   label=L"$u^\mathrm{num}$ (non-conservative)")
    plt.plot(x, u_num_relaxation, label=L"$u^\mathrm{num}$ (conservative)")
    plt.xlabel(L"x"); plt.ylabel(L"u")
    plt.legend(loc="center left", bbox_to_anchor=(1.0, 0.5));

    t_baseline = saved_values_baseline.t
    t_relaxation = saved_values_relaxation.t
    mass_ε_baseline   = map(x->x[1], saved_values_baseline.saveval)
    mass_ε_relaxation = map(x->x[1], saved_values_relaxation.saveval)
    mass_u_baseline   = map(x->x[2], saved_values_baseline.saveval)
    mass_u_relaxation = map(x->x[2], saved_values_relaxation.saveval)
    energy_baseline   = map(x->x[3], saved_values_baseline.saveval)
    energy_relaxation = map(x->x[3], saved_values_relaxation.saveval)

    fig_invariants, ax = plt.subplots(1, 1)
    ax.set_yscale("symlog", linthreshy=1.0e-14)
    plt.plot(t_baseline,   mass_ε_baseline   .- mass_ε_baseline[1],   
        label=L"$\int \varepsilon$ (non-conservative)", color="#E69F00", linestyle="-")
    plt.plot(t_relaxation, mass_ε_relaxation .- mass_ε_relaxation[1], 
        label=L"$\int \varepsilon$ (conservative)", color="#56B4E9", linestyle="-")
    plt.plot(t_baseline,   mass_u_baseline   .- mass_u_baseline[1],   
        label=L"$\int u$ (non-conservative)", color="#E69F00", linestyle="--")
    plt.plot(t_relaxation, mass_u_relaxation .- mass_u_relaxation[1],
        label=L"$\int u$ (conservative)", color="#56B4E9", linestyle="--")
    plt.plot(t_baseline,   energy_baseline   .- energy_baseline[1],   
        label="Energy (non-conservative)", color="#E69F00", linestyle=":")
    plt.plot(t_relaxation, energy_relaxation .- energy_relaxation[1], 
        label="Energy (conservative)", color="#56B4E9", linestyle=":")
    plt.xlabel(L"t"); plt.ylabel("Change of Invariants")
    plt.legend(loc="center left", bbox_to_anchor=(1.0, 0.5))
    
    error_ε_baseline   = map(x->x[4], saved_values_baseline.saveval)
    error_ε_relaxation = map(x->x[4], saved_values_relaxation.saveval)
    error_u_baseline   = map(x->x[5], saved_values_baseline.saveval)
    error_u_relaxation = map(x->x[5], saved_values_relaxation.saveval)

    fig_error1, ax = plt.subplots(1, 1)
    if num_periods < 50
        ax.set_prop_cycle(marker_cycler)
    end
    ax.set_xscale("log")
    ax.set_yscale("log")
    plt.plot(t_baseline,   error_ε_baseline,   label=L"Error of $\varepsilon$ (non-conservative)",
        color="#E69F00", linestyle="-")
    plt.plot(t_relaxation, error_ε_relaxation, label=L"Error of $\varepsilon$ (conservative)",
        color="#56B4E9", linestyle="-")
    plt.plot(t_baseline,   error_u_baseline,   label=L"Error of $u$ (non-conservative)",
        color="#E69F00", linestyle="--")
    plt.plot(t_relaxation, error_u_relaxation, label=L"Error of $u$ (conservative)",
        color="#56B4E9", linestyle="--")
    plt.xlabel(L"t"); plt.ylabel("Error")
    plt.legend(loc="center left", bbox_to_anchor=(1.0, 0.5))
    
    error_σ_baseline    = map(x->x[6], saved_values_baseline.saveval)
    error_σ_relaxation  = map(x->x[6], saved_values_relaxation.saveval)
    error_ϱu_baseline   = map(x->x[7], saved_values_baseline.saveval)
    error_ϱu_relaxation = map(x->x[7], saved_values_relaxation.saveval)

    fig_error2, ax = plt.subplots(1, 1)
    if num_periods < 50
        ax.set_prop_cycle(marker_cycler)
    end
    ax.set_xscale("log")
    ax.set_yscale("log")
    plt.plot(t_baseline,   error_σ_baseline,    label=L"Error of $\sigma$ (non-conservative)",
        color="#E69F00", linestyle="-")
    plt.plot(t_relaxation, error_σ_relaxation,  label=L"Error of $\sigma$ (conservative)",
        color="#56B4E9", linestyle="-")
    plt.plot(t_baseline,   error_ϱu_baseline,   label=L"Error of $\varrho u$ (non-conservative)",
        color="#E69F00", linestyle="--")
    plt.plot(t_relaxation, error_ϱu_relaxation, label=L"Error of $\varrho u$ (conservative)",
        color="#56B4E9", linestyle="--")
    plt.xlabel(L"t"); plt.ylabel("Error")
    plt.legend(loc="center left", bbox_to_anchor=(1.0, 0.5))

    fig_coef, ax = plt.subplots(1, 1)
    plt.plot(x, ϱ.(x), label=L"\varrho")
    plt.xlabel(L"x"); plt.ylabel("Variable coefficients")
    plt.legend(loc="center left", bbox_to_anchor=(1.0, 0.5));
        
    (;sol_baseline, sol_relaxation, saved_values_baseline, saved_values_relaxation, 
        fig_ε, fig_σ, fig_u, fig_invariants, fig_error1, fig_error2, fig_coef)
end


# Error growth of numerically generated solitary waves

In [ ]:
# using a stegoton generated numerically
function mean_to_center(q)
    nghost = 2
    qBCs = similar(q, length(q) + 2*nghost)
    qBCs[1:nghost] = q[end:-1:end-nghost+1]
    qBCs[nghost+1:end-nghost] = q
    qBCs[end-nghost+1:end] = q[1:nghost]
    res = @views @. (9*qBCs[1:end-4] - 116*qBCs[2:end-3] + 2134*qBCs[3:end-2] - 116*qBCs[4:end-1] + 9*qBCs[5:end]) / 1920.0
end

data_filename = joinpath(dirname(@__DIR__), "data", "psystem_refn3_small_domain.h5")
x_center, ε_center, ϱu_center = h5open(data_filename, "r") do io
    read(io, "x"), read(io, "strain"), read(io, "vel")
end

u_center = @. ϱu_center / ϱ_exp(x_center)
dx = (x_center[end] - x_center[1]) / (length(x_center) - 1)
xmin = x_center[1]; xmax = x_center[end]
x = range(xmin, xmax, length=length(x_center))

ε0itp = CubicSplineInterpolation((x,), mean_to_center(ε_center), extrapolation_bc=Periodic())
u0itp = CubicSplineInterpolation((x,), mean_to_center(u_center), extrapolation_bc=Periodic())

get_c() = 0.9769660271402583 # fine grid estimate
function ε_sol(t, x)
    xmin = 0.0
    xmax = 20.0
    c = get_c()
    x_t = mod(x - c*t - xmin, xmax - xmin) + xmin
    ε0itp(x_t)::Float64
end
function u_sol(t, x)
    xmin = 0.0
    xmax = 20.0
    c = get_c()
    x_t = mod(x - c*t - xmin, xmax - xmin) + xmin
    u0itp(x_t)::Float64
end

In [ ]:
@show xmin
@show xmax
@show ε_sol(0., xmin)
@show ε_sol(0., xmax)
@show u_sol(0., xmin)
@show u_sol(0., xmax)
@show N = 2^8
@show dt = 1.0e-1 * (xmax - xmin) / N

@show num_periods = 350
@show tspan = (0., num_periods * (xmax - xmin) / get_c())
flush(stdout)

tol = 1.0e-6
adaptive = true
D = fourier_derivative_operator(xmin, xmax, N)

results = solve_ode_psystem_periodic(ε_sol, u_sol, D, σ_exp, ϱ_exp, η_exp, tspan, 
                                     Tsit5(), tol, dt, adaptive, num_periods);

In [ ]:
ax = results.fig_error1.axes[1]
t = [1.0e3, 8.0e3]
ax.plot(t, 1.5e-8 .* t.^2, ":", color="gray")
ax.annotate(L"\mathcal{O}(t^{2})", (1.0e3, 1.5e-1), color="gray")
ax.plot(t, 3.0e-6 .* t.^1, ":", color="gray")
ax.annotate(L"\mathcal{O}(t^{1})", (1.0e3, 2.0e-3), color="gray")
results.fig_error1.savefig(
    joinpath(dirname(@__DIR__), "figures", "p_system_Fourier_error_epsilon_u.pdf"),
    bbox_inches="tight")
results.fig_error2.savefig(
    joinpath(dirname(@__DIR__), "figures", "p_system_Fourier_error_sigma_rhou.pdf"),
    bbox_inches="tight")

In [ ]:
ε0 = results.sol_baseline[1].x[1]
u0 = results.sol_baseline[1].x[2]
ε_num_baseline   = results.sol_baseline[end].x[1]
u_num_baseline   = results.sol_baseline[end].x[2]
ε_num_relaxation = results.sol_relaxation[end].x[1]
u_num_relaxation = results.sol_relaxation[end].x[2]
x = grid(results.sol_baseline.prob.p.D)
ε_ana = ε_sol.(tspan[end], x)
u_ana = u_sol.(tspan[end], x)

fig_ε, ax = plt.subplots(1, 1)
plt.plot(x, ε_num_baseline,   label="non-conservative")
plt.plot(x, ε_num_relaxation, label="conservative")
plt.plot(x, ε0, ":", color="gray", label="reference")
plt.xlabel(L"x"); plt.ylabel(L"\varepsilon")
plt.xlim(xmin, xmax)
plt.savefig(joinpath(dirname(@__DIR__), "figures", "p_system_Fourier_solution_epsilon.pdf"), bbox_inches="tight")

fig_u, ax = plt.subplots(1, 1)
plt.plot(x, u_num_baseline)
plt.plot(x, u_num_relaxation)
plt.plot(x, u0, ":", color="gray", label="erference")
plt.xlabel(L"x"); plt.ylabel(L"u")
plt.xlim(xmin, xmax)
plt.savefig(joinpath(dirname(@__DIR__), "figures", "p_system_Fourier_solution_u.pdf"), bbox_inches="tight")

plt.figure()
handles, labels = fig_ε.axes[1].get_legend_handles_labels()
plt.figlegend(handles, labels, loc="center", ncol=3)
plt.savefig(joinpath(dirname(@__DIR__), "figures", "p_system_Fourier_solution_legend.pdf"), bbox_inches="tight")